In [65]:
import pandas as pd
import numpy as np
import requests
import logging
import json

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if pd.isna(obj):
            return ''
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, np.int64, np.uint8, np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.bool_)):
            return bool(obj)
        return super(CustomJSONEncoder, self).default(obj)

def csv_row_to_json(csv_file, quote_value):
    try:
        df = pd.read_csv(csv_file)
        row = df[df['QUOTE'] == quote_value].iloc[0]
        return row.to_dict()
    except IndexError:
        logger.error(f"Quote {quote_value} not found in the CSV file.")
        return None
    except Exception as e:
        logger.error(f"Error reading CSV file: {str(e)}")
        return None

def make_prediction_request(data):
    url = "http://localhost:8080/predict"
    headers = {"Content-Type": "application/json"}
    
    try:
        # Convert data to JSON, representing null as empty fields
        json_data = json.dumps({"features": data}, cls=CustomJSONEncoder, separators=(',', ':'))
        
        # Remove quotes from empty fields
        json_data = json_data.replace('""', '')
        
        # Log the payload for debugging
        logger.debug(f"Request payload: {json_data}")
        
        response = requests.post(url, data=json_data, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error making prediction request: {str(e)}")
        logger.error(f"Response content: {response.content if 'response' in locals() else 'No response'}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"Error decoding JSON: {str(e)}")
        return None

# config
train_data_path = '../data/train_raw.csv'
quote = 'FT13323'

json_data = csv_row_to_json(train_data_path, quote)
if json_data:
    # logger.info("JSON data:")
    json_str = json.dumps(json_data, cls=CustomJSONEncoder, indent=2, separators=(',', ': '))
    json_str = json_str.replace('""', '')
    # logger.info(json_str)

    logger.info("\nMaking prediction request...")
    prediction = make_prediction_request(json_data)
    if prediction:
        logger.info("Prediction result:")
        logger.info(json.dumps(prediction, indent=2))
    else:
        logger.error("Failed to get prediction.")
else:
    logger.error("Failed to get JSON data from CSV.")

/var/folders/wh/0n9ffgc50rlgjmhfj0m9s62h0000gn/T/ipykernel_2072/259504508.py:24: DtypeWarning: Columns (38,39,40,42,43,44,45,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,87,88,89,90,91,92,93,94,95,96,97,98,121,130,144,149,153,162) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
INFO:__main__:
Making prediction request...
DEBUG:__main__:Request payload: {"features":{"Unnamed: 0":9459,"QUOTE":"FT13323","CONSUMER_CREDIT_JUDGEMENTS_GUAR1":NaN,"CONSUMER_CREDIT_INSOLVENCY_NOTICES_GUAR1":NaN,"CONSUMER_CREDIT_CREDIT_DEFAULTS_GUAR1":3.0,"CONSUMER_CREDIT_COMPANY_AFFILIATIONS_GUAR1":7.0,"CONSUMER_CREDIT_FILE_ACTIVITY_GUAR1":4.0,"CONSUMER_CREDIT_SCORE_GUAR1":238.0,"CONSUMER_CREDIT_RISK_ODDS_GUAR1":1.0,"CONSUMER_CREDIT_JUDGEMENTS_GUAR2":NaN,"CONSUMER_CREDIT_INSOLVENCY_NOTICES_GUAR2":NaN,"CONSUMER_CREDIT_CREDIT_DEFAULTS_GUAR2":NaN,"CONSUMER_CREDIT_COMPANY_AFFILIATIONS_GUAR2":NaN,"CONSUMER_CREDIT